# Lectura 57: Gráficas en Polars con Matplotlib y Plotly

In [ ]:
import polars as pl
from polars import col

vuelos_lf = pl.scan_parquet('./data/vuelos/vuelos.parquet')

aerolinea_vs_tiempo_aire = vuelos_lf.group_by('AIRLINE').agg(
    AVG_AIR_TIME=col('AIR_TIME').mean()
).collect()

departure_delay_AA = (vuelos_lf.collect()
    .pivot(
        index='AIRLINE',
        columns='MONTH',
        values='DEPARTURE_DELAY',
        aggregate_function='mean'
    )
    .filter(col('AIRLINE') == 'AA')
    .transpose(column_names=['AVG_DEPARTURE_DELAY'])[1:]
    .with_row_index(offset=1)
    .select(
        col('index'),
        col('AVG_DEPARTURE_DELAY').cast(pl.Float64)
    )
)

# Matplotlib

Veamos a continuación como podemos crear un gráfico de barras. Podemos pasar columnas de un `DataFrame` directamente a Matplotlib como una serie para cada columna. Matplotlib no tiene soporte explícito para objetos Polars, pero Matplotlib puede aceptar una serie de Polars porque puede convertir cada serie en un array de numpy.

In [ ]:
import matplotlib.pyplot as plt

plt.bar(
    x=aerolinea_vs_tiempo_aire['AIRLINE'],
    height=aerolinea_vs_tiempo_aire['AVG_AIR_TIME']
)

plt.xlabel('Aerolínea')
plt.ylabel('Promedio tiempo en el aire')
plt.title('Promedio de tiempo en el aire por aerolínea')

Veamos otro ejemplo donde graficaremos el promedio de los retrasos en las salidas de los vuelos por mes en la aerolínea American Airlines.

In [ ]:
plt.plot(
    departure_delay_AA['index'],
    departure_delay_AA['AVG_DEPARTURE_DELAY'],
    'D-r'
)

plt.xlabel('MES')
plt.ylabel('Promedio de retrasos en salidas')
plt.title('Promedio de retrasos en salidas American Airline')

## Plotly

Plotly puede aceptar un DataFrame de Polars aprovechando el protocolo de intercambio de dataframes, el cual ofrece conversión sin copia siempre que sea posible.

In [ ]:
import plotly.express as px

In [ ]:
px.bar(
    aerolinea_vs_tiempo_aire,
    x='AIRLINE',
    y='AVG_AIR_TIME'
)

Realicemos otro ejemplo más y esta vez vamos a personalizar un poco más el gráfico agregándole un título y definiendo marcadores para los puntos.

In [ ]:
px.line(
    aerolinea_vs_tiempo_aire,
    x='AIRLINE',
    y='AVG_AIR_TIME',
    markers=True,
    title='Promedio de retrasos en salidas American Airline'
)